# Chapter 8. Data Wrangling: Join, Combine, and Reshape

## 8.1 Hierarchial Indexing

**Hierarchical indexing enables you to have multiple(two ot more) index levels on an axis**

In [2]:
data=pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])

In [3]:
data

a  1   -0.757125
   2    0.153694
   3    1.308391
b  1    1.196059
   3   -0.802251
c  1    0.230151
   2    0.100702
d  2   -0.467247
   3    0.499487
dtype: float64

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

**Hierachical indexing enables concisely select subset of data**

In [5]:
data['b']

1    1.196059
3   -0.802251
dtype: float64

In [6]:
data['b':'c']

b  1    1.196059
   3   -0.802251
c  1    0.230151
   2    0.100702
dtype: float64

In [7]:
data.loc[['b','d']]

b  1    1.196059
   3   -0.802251
d  2   -0.467247
   3    0.499487
dtype: float64

**Selection from inner level**

In [8]:
data.loc[:,2]

a    0.153694
c    0.100702
d   -0.467247
dtype: float64

unstack **:Rearrange hierarchical data into a DataFrame**

In [9]:
data.unstack()

,1,2,3
a,-0.757125,0.153694,1.308391
b,1.196059,NaN,-0.802251
c,0.230151,0.100702,NaN
d,NaN,-0.467247,0.499487


stack **:Inverse of** unstack

In [10]:
data.unstack().stack()

a  1   -0.757125
   2    0.153694
   3    1.308391
b  1    1.196059
   3   -0.802251
c  1    0.230151
   2    0.100702
d  2   -0.467247
   3    0.499487
dtype: float64

**Either axis in a DataFrame can have a hierarchical index**

In [11]:
frame=pd.DataFrame(np.arange(12).reshape(4,3),index=[['a','a','b','b'],[1,2,1,2],],
                    columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])

In [12]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

**Name hierarchical index**

In [13]:
frame.index.names=['key1','key2']

In [14]:
frame.columns.names=['state','color']

In [15]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

**Partial column indexing can be used to select groups of columns**

In [16]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

MultiIndex **can be created by itself and reused**

In [20]:
pd.MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],['Green','Red','Green']],
                         names=['state','color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

swaplevel **takes two level numbers or names and returns a new object with the levels interchanges**

In [21]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

sort_index **sorts the data using only the values in a single level**

In [26]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

sort_index **Combine swapping level with sort_index**

In [23]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

**Many descriptive and summary statistics have a** level **option to specify the particular level**

In [27]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [28]:
frame.sum(level='color',axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's Columns

In [29]:
frame=pd.DataFrame({'a':range(7),'b':range(7,0,-1),'c':['one','one','one','two','two','two','two'],
                   'd':[0,1,2,0,1,2,3]})

In [30]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


set_index **function will create a new DataFrame using one or more of its columns as the index**

In [34]:
frame2=frame.set_index(['c','d'])#by default, columns are removed from the DataFrame
#use drop=False

In [32]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [35]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combining and Merging Datasets

pandas.merge **connects rows in DataFrame based on one or more keys**<br>
pandas.concat **concatenates or 'stacks' together objects along an axis**<br>
combine_first **enables splicing together overlapping data to fill in missing values**

### Database-Style DataFrame Joins

***Merge* or *join* operations combin datasets by linking rows using one or more keys**

In [39]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [40]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


**One-to-Many Join**

In [41]:
pd.merge(df1,df2)#merge use the overlapping column names as the keys by default

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [45]:
df3

,1key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [46]:
df4

,rkey,data1
0,a,0
1,b,1
2,d,2


In [47]:
pd.merge(df3,df4,left_on='1key',right_on='rkey')#columns names are different in each object, 
#need to specify them separately

,1key,data1_x,rkey,data1_y
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


**By default,** merge **does an inner join**<br>
**other possible options are** 'left', 'right', 'outer'

In [48]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


**Many-to-many joins form the Cartesian prodcut of the rows**
**the join method only affects the distinct key values**

In [50]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [52]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [60]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [58]:
pd.merge(df1,df2,how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


**Merge with multiple keys, pass a list of column names**

In [66]:
left

,key1,key2,lval
0,foo,one,1
1,foo,one,2
2,bar,one,3


In [68]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [69]:
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4
1,foo,one,1.0,5
2,foo,one,2.0,4
3,foo,one,2.0,5
4,bar,one,3.0,6
5,bar,two,NaN,7


**Overlapping column names**<br>
merge **has a** suffixes **option for specifying strings to append to overlapping names in the left and right DataFram object**

In [71]:
pd.merge(left,right,on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,one,2,one,4
3,foo,one,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

**Pass** left_index=True **or** right_index=True **to indciate that the indeex should be used as a merge key**

In [76]:
left1=pd.DataFrame({'key':['a','b','a','a','b','c'],'value':range(6)})

In [129]:
right1=pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])

In [130]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [131]:
right1

,group_val
a,3.5
b,7.0


In [132]:
%%bash
git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    .ipynb_checkpoints/Untitled-checkpoint.ipynb
	deleted:    .ipynb_checkpoints/Untitled1-checkpoint.ipynb
	modified:   Data_Loading_Practice.ipynb
	deleted:    Untitled.ipynb
	deleted:    Untitled1.ipynb
	modified:   data_cleaning.ipynb
	modified:   datawrangling.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.DS_Store
	.ipynb_checkpoints/Data_Loading_Practice-checkpoint.ipynb
	.ipynb_checkpoints/data_cleaning-checkpoint.ipynb
	.ipynb_checkpoints/datawrangling-checkpoint.ipynb
	.ipynb_checkpoints/practice1-checkpoint.ipynb
	Book1.xlsx
	practice1.ipynb
	transaction.csv

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
%%bash
git add datawrangling